## Set up code

Run cells under "Sped up" and "saver" before beginning.

### Original

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

def get_delta_E(J, sigma):
    return sigma * (2*J @ sigma)

def generate_D(lambda_param):
    return -np.random.exponential(scale = 1/lambda_param) 

def flip_i_star(delta_E, D, sigma):
    neg_indices = np.where(delta_E < 0)[0]
    if neg_indices.size > 0:
        i_star = neg_indices[np.argmin(np.abs(delta_E[neg_indices] - D))]
        sigma[i_star] *= -1
    else:
        assert False
    return sigma

def flip_i_star_greedy(delta_E, sigma):
    neg_indices = np.where(delta_E < 0)[0]
    if neg_indices.size > 0:
        i_star = neg_indices[np.argmax(np.abs(delta_E[neg_indices]))]
        sigma[i_star] *= -1
    else:
        assert False
    return sigma

def flip_i_star_reluctant(delta_E, sigma):
    neg_indices = np.where(delta_E < 0)[0]
    if neg_indices.size > 0:
        i_star = neg_indices[np.argmin(np.abs(delta_E[neg_indices]))]
        sigma[i_star] *= -1
    else:
        assert False
    return sigma

def evolution_rule(J, sigma, lambda_param, max_iter = 10000000000):
    for t in range(max_iter): 
        delta_E = get_delta_E(J, sigma)
        if np.all(delta_E >= 0):
            return sigma, t
        D = generate_D(lambda_param) 
        sigma = flip_i_star(delta_E, D, sigma)
    
    print("Maximum iterations reached without finding a local minimum")
    return sigma, max_iter

def evolution_rule_extreme(J, sigma, dynamic, max_iter = 10000000000):
    if dynamic == "greedy":
        for t in range(max_iter): 
            delta_E = get_delta_E(J, sigma)
            if np.all(delta_E >= 0):
                return sigma, t
            sigma = flip_i_star_greedy(delta_E, sigma)
        print("Maximum iterations reached without finding a local minimum")
        return sigma, max_iter
    elif dynamic == "reluctant":
        for t in range(max_iter): 
            delta_E = get_delta_E(J, sigma)
            if np.all(delta_E >= 0):
                return sigma, t
            sigma = flip_i_star_reluctant(delta_E, sigma)
        print("Maximum iterations reached without finding a local minimum")
        return sigma, max_iter        

def run_simulation(N, lambda_param = None, dynamic = None, nreal = 1000):
    time = np.zeros(nreal)
    min_energy = np.zeros(nreal)
    
    if dynamic == "greedy":
        for i in range(nreal):
            J, sigma = initialize_system(N)
            final_sigma, iterations = evolution_rule_extreme(J, np.copy(sigma), dynamic)
            time[i] = iterations
            min_energy[i] = -0.5 * final_sigma.T @ J @ final_sigma / N
    
    elif dynamic == "reluctant":
        for i in tqdm(range(nreal), desc="Running simulations"):
            J, sigma = initialize_system(N)
            final_sigma, iterations = evolution_rule_extreme(J, np.copy(sigma), dynamic)
            time[i] = iterations
            min_energy[i] = -0.5 * final_sigma.T @ J @ final_sigma / N
    
    else:
        for i in range(nreal):
            J, sigma = initialize_system(N)
            final_sigma, iterations = evolution_rule(J, np.copy(sigma), lambda_param)
            time[i] = iterations
            min_energy[i] = -0.5 * final_sigma.T @ J @ final_sigma / N

    log_time = np.log10(time)
    avg_time = np.mean(time)
    avg_min_energy = np.mean(min_energy)
    std_time = np.std(time)
    std_energy = np.std(min_energy)
    return avg_time, avg_min_energy, std_time, std_energy, log_time

def run_experiment():
    # greedy -> reluctant
    #lambda_values = [1, 10, 25, 45, 70, 100]
    lambda_values = [1, 10, 100]
    #N_values = [25, 40, 50, 100, 150, 200, 300]
    N_values = [100, 250, 500, 750, 1000, 1500]

    results_time = np.zeros((len(N_values), len(lambda_values)+2))
    results_energy = np.zeros((len(N_values), len(lambda_values)+2))
    std_time = np.zeros((len(N_values), len(lambda_values)+2))
    std_energy = np.zeros((len(N_values), len(lambda_values)+2))
    log_time = np.zeros((len(N_values), len(lambda_values)+2, 1000))

    for i in range(len(N_values)):
        for j in range(len(lambda_values)):
            print("\nN: ", N_values[i], "\tlambda: ", lambda_values[j])
            results_time[i][j], results_energy[i][j], std_time[i, j], std_energy[i, j], log_time[i, j] = run_simulation(N_values[i], lambda_values[j])
        print("\nN: ", N_values[i], "\tgreedy")
        results_time[i][j+1], results_energy[i][j+1], std_time[i, j+1], std_energy[i, j+1], log_time[i, j+1] = run_simulation(N_values[i], dynamic="greedy")
        print("\nN: ", N_values[i], "\treluctant")
        results_time[i][j+2], results_energy[i][j+2], std_time[i, j+2], std_energy[i, j+2], log_time[i, j+2] = run_simulation(N_values[i], dynamic="reluctant")

    return results_time, results_energy, std_time, std_energy, log_time

### Sped up version

In [2]:
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm

# ----------------------
# Utilities
# ----------------------

def get_initial_delta_E(J, sigma):
    return sigma * (2 * (J @ sigma))

def generate_D(lambda_param):
    return -np.random.exponential(scale=1/lambda_param)

def flip_update(J, sigma, delta_E, i_star, debug=False):
    s_k = sigma[i_star]
    sigma[i_star] = -s_k

    # Update all other spins incrementally
    for j in range(len(sigma)):
        if j != i_star:
            delta_E[j] -= 4 * J[i_star, j] * sigma[j] * s_k

    # Flip delta E for the chosen spin
    delta_E[i_star] = -delta_E[i_star]

    if debug:
        delta_E_true = sigma * (2 * (J @ sigma))
        max_diff = np.max(np.abs(delta_E - delta_E_true))
        if max_diff > 1e-8:   # tolerate floating-point noise
            print(f"[DEBUG] max ΔE mismatch = {max_diff:.2e}")

    return sigma, delta_E

# ----------------------
# Dynamics
# ----------------------

def evolution_rule_fast(J, sigma, lambda_param, max_iter=1_000_000):
    delta_E = get_initial_delta_E(J, sigma)
    for t in range(max_iter):
        if np.all(delta_E >= 0):
            return sigma, t

        D = generate_D(lambda_param)
        neg_indices = np.where(delta_E < 0)[0]
        if neg_indices.size == 0:
            return sigma, t

        i_star = neg_indices[np.argmin(np.abs(delta_E[neg_indices] - D))]
        sigma, delta_E = flip_update(J, sigma, delta_E, i_star, debug = True)

    print("Maximum iterations reached without finding a local minimum")
    return sigma, max_iter


def evolution_rule_extreme_fast(J, sigma, dynamic, max_iter=1_000_000):
    delta_E = get_initial_delta_E(J, sigma)
    for t in range(max_iter):
        if np.all(delta_E >= 0):
            return sigma, t

        neg_indices = np.where(delta_E < 0)[0]
        if neg_indices.size == 0:
            return sigma, t

        if dynamic == "greedy":
            i_star = neg_indices[np.argmax(np.abs(delta_E[neg_indices]))]
        elif dynamic == "reluctant":
            i_star = neg_indices[np.argmin(np.abs(delta_E[neg_indices]))]
        else:
            raise ValueError("dynamic must be 'greedy' or 'reluctant'")

        sigma, delta_E = flip_update(J, sigma, delta_E, i_star, debug = True)

    print("Maximum iterations reached without finding a local minimum")
    return sigma, max_iter

# ----------------------
# Simulation
# ----------------------

def single_run(N, lambda_param=None, dynamic=None):
    J, sigma = initialize_system(N)
    if dynamic in ("greedy", "reluctant"):
        final_sigma, iterations = evolution_rule_extreme_fast(J, np.copy(sigma), dynamic)
    else:
        final_sigma, iterations = evolution_rule_fast(J, np.copy(sigma), lambda_param)

    time = iterations
    min_energy = -0.5 * final_sigma.T @ J @ final_sigma / N
    return time, min_energy

#n_jobs = -1 runs them all. I have 10 cores.
def run_simulation(N, lambda_param=None, dynamic=None, nreal=1000, n_jobs=6, show_progress=True):
    iterator = range(nreal)
    if show_progress and dynamic == "reluctant":
        iterator = tqdm(iterator, desc=f"Running simulations N={N}, dynamic={dynamic}")

    results = Parallel(n_jobs=n_jobs)(
        delayed(single_run)(N, lambda_param, dynamic) for _ in iterator
    )

    time, min_energy = zip(*results)
    time = np.array(time)
    min_energy = np.array(min_energy)

    log_time = np.log10(time)
    avg_time = np.mean(time)
    avg_min_energy = np.mean(min_energy)
    std_time = np.std(time)
    std_energy = np.std(min_energy)
    return avg_time, avg_min_energy, std_time, std_energy, log_time

def run_experiment():
    # greedy -> reluctant
    #lambda_values = [1, 10, 25, 45, 70, 100]
    lambda_values = [1, 10, 100]

    #N_values = [25, 40, 50, 100, 150, 200, 300]
    N_values = [100, 200, 300, 400, 500, 750, 1000]

    results_time = np.zeros((len(N_values), len(lambda_values)+2))
    results_energy = np.zeros((len(N_values), len(lambda_values)+2))
    std_time = np.zeros((len(N_values), len(lambda_values)+2))
    std_energy = np.zeros((len(N_values), len(lambda_values)+2))
    log_time = np.zeros((len(N_values), len(lambda_values)+2, 1000))

    for i in range(len(N_values)):
        for j in range(len(lambda_values)):
            print("\nN: ", N_values[i], "\tlambda: ", lambda_values[j])
            results_time[i][j], results_energy[i][j], std_time[i, j], std_energy[i, j], log_time[i, j] = run_simulation(N_values[i], lambda_values[j])
        print("\nN: ", N_values[i], "\tgreedy")
        results_time[i][j+1], results_energy[i][j+1], std_time[i, j+1], std_energy[i, j+1], log_time[i, j+1] = run_simulation(N_values[i], dynamic="greedy")
        print("\nN: ", N_values[i], "\treluctant")
        results_time[i][j+2], results_energy[i][j+2], std_time[i, j+2], std_energy[i, j+2], log_time[i, j+2] = run_simulation(N_values[i], dynamic="reluctant")

    return results_time, results_energy, std_time, std_energy, log_time

### Saver

In [3]:
import pandas as pd
def save_results(results_time, results_energy, std_time, std_energy, log_time, name):
        #lambda_values = [1, 10, 25, 45, 70, 100]

        lambda_values = [1, 10, 100]

        # N_values = [25, 40, 50, 100, 150, 200, 300]
        N_values = [100, 200, 300, 400, 500, 750, 1000]


        data = []

        for i in range(results_time.shape[0]):  # Loop over N values
            for j in range(results_time.shape[1] - 2):  # Loop over lambda values
                data.append([
                    N_values[i],
                    lambda_values[j],
                    results_time[i, j],
                    results_energy[i, j], 
                    std_time[i, j],
                    std_energy[i, j], 
                    log_time[i, j].tolist(), 
                    np.mean(log_time[i, j].tolist())
                ])
            data.append([
                N_values[i],
                0,
                results_time[i, j+1],
                results_energy[i, j+1], 
                std_time[i, j+1],
                std_energy[i, j+1], 
                log_time[i, j+1].tolist(), 
                np.mean(log_time[i, j+1].tolist())
            ])
            data.append([
                N_values[i],
                -1,
                results_time[i, j+2],
                results_energy[i, j+2], 
                std_time[i, j+2],
                std_energy[i, j+2], 
                log_time[i, j+2].tolist(), 
                np.mean(log_time[i, j+2].tolist())
            ])
        df = pd.DataFrame(data, columns=['N', 'Lambda', 'Time', 'Energy', 'std_Time', 'std_Energy', 'log_time', 'avg_log_time'])
        df.to_csv(name + '.csv', index=False)
        return df


## Gaussian

In [10]:
def initialize_J_normal(N):
    J = np.random.normal(0, 1/np.sqrt(N), size=(N, N))
    for i in range(N):
        for j in range(i + 1, N):
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0) 
    return J

def initialize_system(N):
    J = initialize_J_normal(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [11]:
results_time1, results_energy1, std_time1, std_energy1, log_time1 = run_experiment()
df = save_results(results_time1, results_energy1, std_time1, std_energy1, log_time1, "j_norm_dist_NEW")


N:  100 	lambda:  1

N:  100 	lambda:  10

N:  100 	lambda:  100

N:  100 	greedy

N:  100 	reluctant


Running simulations N=100, dynamic=reluctant: 100%|██████████| 1000/1000 [01:24<00:00, 11.79it/s]



N:  200 	lambda:  1

N:  200 	lambda:  10

N:  200 	lambda:  100

N:  200 	greedy

N:  200 	reluctant


Running simulations N=200, dynamic=reluctant: 100%|██████████| 1000/1000 [27:34<00:00,  1.65s/it] 



N:  300 	lambda:  1

N:  300 	lambda:  10

N:  300 	lambda:  100

N:  300 	greedy

N:  300 	reluctant


Running simulations N=300, dynamic=reluctant: 100%|██████████| 1000/1000 [1:49:41<00:00,  6.58s/it]   



N:  400 	lambda:  1

N:  400 	lambda:  10


KeyboardInterrupt: 

## Uniform {-1, +1}

In [3]:
def initialize_J_unif(N):
    J = np.random.choice([-1/np.sqrt(N), 1/np.sqrt(N)], size=(N, N))
    for i in range(N):
        for j in range(i + 1, N):
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0) 
    return J

def initialize_system(N):
    J = initialize_J_unif(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [4]:
results_time2, results_energy2, std_time2, std_energy2, log_time2 = run_experiment()
df = save_results(results_time2, results_energy2, std_time2, std_energy2, log_time2, "data/j_unif_dist")



N:  25 	lambda:  1

N:  25 	lambda:  10

N:  25 	lambda:  25

N:  25 	lambda:  45

N:  25 	lambda:  70

N:  25 	lambda:  100

N:  25 	greedy

N:  25 	reluctant

N:  40 	lambda:  1

N:  40 	lambda:  10

N:  40 	lambda:  25

N:  40 	lambda:  45

N:  40 	lambda:  70

N:  40 	lambda:  100

N:  40 	greedy

N:  40 	reluctant

N:  50 	lambda:  1

N:  50 	lambda:  10

N:  50 	lambda:  25

N:  50 	lambda:  45

N:  50 	lambda:  70

N:  50 	lambda:  100

N:  50 	greedy

N:  50 	reluctant

N:  100 	lambda:  1

N:  100 	lambda:  10

N:  100 	lambda:  25

N:  100 	lambda:  45

N:  100 	lambda:  70

N:  100 	lambda:  100

N:  100 	greedy

N:  100 	reluctant

N:  150 	lambda:  1

N:  150 	lambda:  10

N:  150 	lambda:  25

N:  150 	lambda:  45

N:  150 	lambda:  70

N:  150 	lambda:  100

N:  150 	greedy

N:  150 	reluctant

N:  200 	lambda:  1

N:  200 	lambda:  10

N:  200 	lambda:  25

N:  200 	lambda:  45

N:  200 	lambda:  70

N:  200 	lambda:  100

N:  200 	greedy

N:  200 	reluctant

N:  300 	

## Uniform (-sqrt3, +sqrt3)

In [ ]:
def initialize_J_uniform_cont(N):
    scale = np.sqrt(3) / np.sqrt(N)  # ensures variance = 1/N
    J = np.random.uniform(-scale, scale, size=(N, N))
    for i in range(N):
        for j in range(i + 1, N):
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0)
    return J

def initialize_system(N):
    J = initialize_J_uniform_cont(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [ ]:
results_time3, results_energy3, std_time3, std_energy3, log_time3 = run_experiment()
df = save_results(results_time3, results_energy3, std_time3, std_energy3, log_time3, "data/j_cont_unif_dist")

## Laplace

In [ ]:
def initialize_J_laplace(N):
    b = np.sqrt(1 / (2 * N))
    J = np.random.laplace(0, b, size=(N, N))
    for i in range(N):
        for j in range(i + 1, N):
            J[j, i] = J[i, j]  
    np.fill_diagonal(J, 0)  
    return J

def initialize_system(N):
    J = initialize_J_laplace(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [ ]:
results_time4, results_energy4, std_time4, std_energy4, log_time4 = run_experiment()
df = save_results(results_time4, results_energy4, std_time4, std_energy4, log_time4, "data/j_laplace_dist")

## Hyperbolic Secant

In [ ]:
def initialize_J_sec(N):
    J = np.zeros((N, N))
    for i in range(N):
        for j in range(i + 1, N):
            U = np.random.uniform(0, 1)
            X = (2 / np.pi) * np.log(np.tan(np.pi * U / 2))
            J[i, j] = X / np.sqrt(N)
            J[j, i] = J[i, j]
    return J

def initialize_system(N):
    J = initialize_J_sec(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [ ]:
results_time5, results_energy5, std_time5, std_energy5, log_time5 = run_experiment()
df = save_results(results_time5, results_energy5, std_time5, std_energy5, log_time5, "data/j_sec_dist")

## "Trimodal" Distriubtion

In [ ]:
def initialize_J_trimodal(N):
    J = np.random.rand(N, N)
    for i in range(N):
        for j in range(i + 1, N):
            if J[i, j] < 1/6:
                J[i, j] = 1/np.sqrt(N) * np.sqrt(3)
            elif J[i, j] < 1/3:
                J[i, j] = -1/np.sqrt(N) * np.sqrt(3)
            else:
                J[i, j] = 0
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0) 
    return J

def initialize_system(N):
    J = initialize_J_trimodal(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [ ]:
results_time6, results_energy6, std_time6, std_energy6, log_time6 = run_experiment()
df = save_results(results_time6, results_energy6, std_time6, std_energy6, log_time6, "../large_N_data/j_sixthmom_dist")


N:  25 	lambda:  1

N:  25 	lambda:  10

N:  25 	lambda:  25

N:  25 	lambda:  45

N:  25 	lambda:  70

N:  25 	lambda:  100

N:  25 	greedy

N:  25 	reluctant

N:  40 	lambda:  1

N:  40 	lambda:  10

N:  40 	lambda:  25

N:  40 	lambda:  45

N:  40 	lambda:  70

N:  40 	lambda:  100

N:  40 	greedy

N:  40 	reluctant

N:  50 	lambda:  1

N:  50 	lambda:  10

N:  50 	lambda:  25

N:  50 	lambda:  45

N:  50 	lambda:  70

N:  50 	lambda:  100

N:  50 	greedy

N:  50 	reluctant

N:  100 	lambda:  1

N:  100 	lambda:  10

N:  100 	lambda:  25

N:  100 	lambda:  45

N:  100 	lambda:  70

N:  100 	lambda:  100

N:  100 	greedy

N:  100 	reluctant

N:  150 	lambda:  1

N:  150 	lambda:  10

N:  150 	lambda:  25

N:  150 	lambda:  45

N:  150 	lambda:  70

N:  150 	lambda:  100

N:  150 	greedy

N:  150 	reluctant

N:  200 	lambda:  1

N:  200 	lambda:  10

N:  200 	lambda:  25

N:  200 	lambda:  45

N:  200 	lambda:  70

N:  200 	lambda:  100

N:  200 	greedy

N:  200 	reluctant

N:  300 	

## Gaussian + Sparse

In [10]:
def initialize_J_gaussian_sparse(N, p = 2/3):
    J = np.zeros((N, N))
    std = np.sqrt(1 / (1 - p))  # = sqrt(3) if p=2/3
    
    for i in range(N):
        for j in range(i + 1, N):
            if np.random.rand() > p:
                J_ij = np.random.normal(loc=0, scale=std) / np.sqrt(N)
                J[i, j] = J_ij
                J[j, i] = J_ij

    return J

def initialize_system(N):
    J = initialize_J_gaussian_sparse(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [12]:
results_time7, results_energy7, std_time7, std_energy7, log_time7 = run_experiment()
df = save_results(results_time7, results_energy7, std_time7, std_energy7, log_time7, "data/j_sparsenorm_dist")


N:  25 	lambda:  1

N:  25 	lambda:  10

N:  25 	lambda:  25

N:  25 	lambda:  45

N:  25 	lambda:  70

N:  25 	lambda:  100

N:  25 	greedy

N:  25 	reluctant

N:  40 	lambda:  1

N:  40 	lambda:  10

N:  40 	lambda:  25

N:  40 	lambda:  45

N:  40 	lambda:  70

N:  40 	lambda:  100

N:  40 	greedy

N:  40 	reluctant

N:  50 	lambda:  1

N:  50 	lambda:  10

N:  50 	lambda:  25

N:  50 	lambda:  45

N:  50 	lambda:  70

N:  50 	lambda:  100

N:  50 	greedy

N:  50 	reluctant

N:  100 	lambda:  1

N:  100 	lambda:  10

N:  100 	lambda:  25

N:  100 	lambda:  45

N:  100 	lambda:  70

N:  100 	lambda:  100

N:  100 	greedy

N:  100 	reluctant

N:  150 	lambda:  1

N:  150 	lambda:  10

N:  150 	lambda:  25

N:  150 	lambda:  45

N:  150 	lambda:  70

N:  150 	lambda:  100

N:  150 	greedy

N:  150 	reluctant

N:  200 	lambda:  1

N:  200 	lambda:  10

N:  200 	lambda:  25

N:  200 	lambda:  45

N:  200 	lambda:  70

N:  200 	lambda:  100

N:  200 	greedy

N:  200 	reluctant

N:  300 	

## Irrational

In [ ]:
def initialize_J_irrat(N):
    J = np.zeros((N, N))
    p_neg1 = np.sqrt(2) - 1 
    p_0 = 1 - (np.sqrt(2) / 2)
    p_root2 = 1 - p_neg1 - p_0
    
    values = [-1/np.sqrt(N), 0, np.sqrt(2)/np.sqrt(N)]
    probs = [p_neg1, p_0, p_root2]
    
    for i in range(N):
        for j in range(i+1, N):
            J[i, j] = np.random.choice(values, p=probs)
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0)

    return J

def initialize_system(N):
    J = initialize_J_irrat(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [ ]:
results_time7, results_energy7, std_time7, std_energy7, log_time7 = run_experiment()
df = save_results(results_time7, results_energy7, std_time7, std_energy7, log_time7, "../large_N_data/j_irrational_dist")


N:  25 	lambda:  1

N:  25 	lambda:  10

N:  25 	lambda:  25

N:  25 	lambda:  45

N:  25 	lambda:  70

N:  25 	lambda:  100

N:  25 	greedy

N:  25 	reluctant


Running simulations: 100%|██████████| 1000/1000 [00:06<00:00, 152.21it/s]



N:  40 	lambda:  1

N:  40 	lambda:  10

N:  40 	lambda:  25

N:  40 	lambda:  45

N:  40 	lambda:  70

N:  40 	lambda:  100

N:  40 	greedy

N:  40 	reluctant


Running simulations: 100%|██████████| 1000/1000 [00:16<00:00, 59.40it/s]



N:  50 	lambda:  1

N:  50 	lambda:  10

N:  50 	lambda:  25

N:  50 	lambda:  45

N:  50 	lambda:  70

N:  50 	lambda:  100

N:  50 	greedy

N:  50 	reluctant


...

Running simulations: 100%|██████████| 1000/1000 [10:11<00:00,  1.64it/s]  



N:  200 	lambda:  1

N:  200 	lambda:  10

N:  200 	lambda:  25

N:  200 	lambda:  45

N:  200 	lambda:  70

N:  200 	lambda:  100

N:  200 	greedy

N:  200 	reluctant


Running simulations: 100%|██████████| 1000/1000 [09:35<00:00,  1.74it/s]



N:  300 	lambda:  1

N:  300 	lambda:  10

N:  300 	lambda:  25

N:  300 	lambda:  45

N:  300 	lambda:  70

N:  300 	lambda:  100

N:  300 	greedy

N:  300 	reluctant


Running simulations: 100%|██████████| 1000/1000 [23:56<00:00,  1.44s/it]


## 5 point that matches to 6th mom (Irrational)

In [4]:
def run_large_N_experiment():
    N_values0 = [25, 40, 50, 100, 150, 200, 300]
    N_values1 = [50, 80, 100, 200, 300, 400, 600]
    N_values2 = [100, 160, 200, 400, 600, 800, 1200]
    N_values3 = [200, 320, 400, 800, 1200, 1600, 2400]
    N_list = [N_values0, N_values1, N_values2]

    max_len = max(len(N_values) for N_values in N_list)

    results_time = np.full((len(N_list), max_len), np.nan)
    results_energy = np.full((len(N_list), max_len), np.nan)
    std_time = np.full((len(N_list), max_len), np.nan)
    std_energy = np.full((len(N_list), max_len), np.nan)
    log_time = np.full((len(N_list), max_len, 1000), np.nan)

    cache = {}
    for g, N_values in enumerate(N_list):
        for i, N in enumerate(N_values):
            if N not in cache:
                print(f"\nRunning simulation for N={N} (Reluctant dynamic)")
                cache[N] = run_simulation(N, dynamic="reluctant")
            else:
                print(f"\nUsing cached result for N={N}")
            results_time[g, i], results_energy[g, i], std_time[g, i], std_energy[g, i], log_time[g, i] = cache[N]
    return results_time, results_energy, std_time, std_energy, log_time


import pandas as pd
def save_large_N_results(results_time, results_energy, std_time, std_energy, log_time, name):
    # Define N lists (same order you used in run_experiment)
    N_values0 = [25, 40, 50, 100, 150, 200, 300]
    N_values1 = [50, 80, 100, 200, 300, 400, 600]
    N_values2 = [100, 160, 200, 400, 600, 800, 1200]
    N_values3 = [200, 320, 400, 800, 1200, 1600, 2400]
    N_list = [N_values0, N_values1, N_values2]

    data = []

    # Loop over groups
    for g, N_values in enumerate(N_list):
        for i, N in enumerate(N_values):
            data.append([
                g,
                N,
                -1,
                results_time[g, i],
                results_energy[g, i],
                std_time[g, i],
                std_energy[g, i],
                log_time[g, i].tolist(),
                np.mean(log_time[g, i].tolist())
            ])

    df = pd.DataFrame(data, columns=['Group', 'N', 'Lambda', 'Time', 'Energy', 'std_Time', 'std_Energy', 'log_time', 'avg_log_time'])
    df.to_csv(name + '.csv', index=False)
    return df


In [5]:
def initialize_J_moment(N):
    J = np.zeros((N, N))
    p_1 = 3/10
    p_0 = 1/3
    p_root6 = 1/30
    
    values = [-1/np.sqrt(N), 1/np.sqrt(N), 0, np.sqrt(6)/np.sqrt(N), -np.sqrt(6)/np.sqrt(N)]
    probs = [p_1, p_1, p_0, p_root6, p_root6]
    
    for i in range(N):
        for j in range(i+1, N):
            J[i, j] = np.random.choice(values, p=probs)
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0)

    return J

def initialize_system(N):
    J = initialize_J_moment(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [6]:
results_time8, results_energy8, std_time8, std_energy8, log_time8 = run_large_N_experiment()
df = save_large_N_results(results_time8, results_energy8, std_time8, std_energy8, log_time8, "../large_N_data/j_moment7_irrat_dist_relonly2")


Running simulation for N=25 (Reluctant dynamic)


Running simulations N=25, dynamic=reluctant:   0%|          | 0/1000 [00:00<?, ?it/s]

Running simulations N=25, dynamic=reluctant: 100%|██████████| 1000/1000 [00:03<00:00, 303.83it/s]



Running simulation for N=40 (Reluctant dynamic)


Running simulations N=40, dynamic=reluctant: 100%|██████████| 1000/1000 [00:03<00:00, 319.73it/s]


...

Running simulations N=160, dynamic=reluctant: 100%|██████████| 1000/1000 [01:13<00:00, 13.54it/s]



Using cached result for N=200

Using cached result for N=400

Using cached result for N=600

Running simulation for N=800 (Reluctant dynamic)


Running simulations N=800, dynamic=reluctant: 100%|██████████| 1000/1000 [1:14:53<00:00,  4.49s/it]



Running simulation for N=1200 (Reluctant dynamic)


Running simulations N=1200, dynamic=reluctant: 100%|██████████| 1000/1000 [7:57:07<00:00, 28.63s/it]   


In [9]:
df = save_large_N_results(results_time8, results_energy8, std_time8, std_energy8, log_time8, "../large_N_data/j_moment7_irrat_dist_relonly")

## 7 point that matches to 6th mom (Rational)

In [18]:
def initialize_J_moment7(N):
    p1 = 1/4
    p2 = 1/20
    p3 = 1/180
    p0 = 14/36

    values = [0, 1/ np.sqrt(N), -1/ np.sqrt(N), 2/ np.sqrt(N), -2/ np.sqrt(N), 3/ np.sqrt(N), -3/ np.sqrt(N)]
    probs = np.array([p0, p1, p1, p2, p2, p3, p3], dtype=float)

    # Number of upper-triangular entries (excluding diagonal)
    num_entries = N * (N - 1) // 2

    # Sample all at once
    samples = np.random.choice(values, size=num_entries, p=probs)

    # Fill into symmetric matrix
    J = np.zeros((N, N), dtype=float)
    triu_indices = np.triu_indices(N, k=1)
    J[triu_indices] = samples
    J[(triu_indices[1], triu_indices[0])] = samples  # mirror

    return J

def initialize_system(N):
    J = initialize_J_moment7(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [19]:
results_time8, results_energy8, std_time8, std_energy8, log_time8 = run_experiment()
df = save_results(results_time8, results_energy8, std_time8, std_energy8, log_time8, "data/j_moment7_rational_dist_100")


N:  25 	lambda:  1

N:  25 	lambda:  10

N:  25 	lambda:  25

N:  25 	lambda:  45

N:  25 	lambda:  70

N:  25 	lambda:  100

N:  25 	greedy

N:  25 	reluctant


Running simulations N=25, dynamic=reluctant: 100%|██████████| 1000/1000 [00:00<00:00, 1124.99it/s]



N:  40 	lambda:  1

N:  40 	lambda:  10

N:  40 	lambda:  25

N:  40 	lambda:  45

N:  40 	lambda:  70

N:  40 	lambda:  100

N:  40 	greedy

N:  40 	reluctant


Running simulations N=40, dynamic=reluctant: 100%|██████████| 1000/1000 [00:01<00:00, 810.16it/s]



N:  50 	lambda:  1

N:  50 	lambda:  10

N:  50 	lambda:  25

N:  50 	lambda:  45

N:  50 	lambda:  70

N:  50 	lambda:  100

N:  50 	greedy

N:  50 	reluctant


Running simulations N=50, dynamic=reluctant: 100%|██████████| 1000/1000 [00:01<00:00, 588.40it/s]



N:  100 	lambda:  1

N:  100 	lambda:  10

N:  100 	lambda:  25

N:  100 	lambda:  45

N:  100 	lambda:  70

N:  100 	lambda:  100

N:  100 	greedy

N:  100 	reluctant


Running simulations N=100, dynamic=reluctant: 100%|██████████| 1000/1000 [00:05<00:00, 189.25it/s]


## 9th moment

In [ ]:
from fractions import Fraction
def initialize_J_moment9(N): 
    p1 = Fraction(29, 120)
    p2 = Fraction(13, 240)
    p3 = Fraction(11, 2520)
    p4 = Fraction(1, 6720)

    # Remaining probability goes to 0
    p0 = 1 - 2*(p1 + p2 + p3 + p4)

    values = [0, 1, -1, 2, -2, 3, -3, 4, -4]
    probs = [p0, p1, p1, p2, p2, p3, p3, p4, p4]

    # Convert to floats for np.random.choice
    probs = np.array([float(p) for p in probs])

    # Number of upper-triangular entries
    num_entries = N * (N - 1) // 2

    # Sample all at once
    samples = np.random.choice(values, size=num_entries, p=probs) / np.sqrt(N)

    # Fill into symmetric matrix
    J = np.zeros((N, N), dtype=float)
    triu_indices = np.triu_indices(N, k=1)
    J[triu_indices] = samples
    J[(triu_indices[1], triu_indices[0])] = samples  # mirror

    return J

def initialize_system(N):
    J = initialize_J_moment9(N)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [9]:
from fractions import Fraction

p1 = Fraction(29, 120)
p2 = Fraction(13, 240)
p3 = Fraction(11, 2520)
p4 = Fraction(1, 6720)

# Remaining probability goes to 0
p0 = 1 - 2*(p1 + p2 + p3 + p4)
print(p0)

115/288


In [ ]:
results_time9, results_energy9, std_time9, std_energy9, log_time9 = run_experiment()
df = save_results(results_time9, results_energy9, std_time9, std_energy9, log_time9, "data/j_moment9_rational_dist")


N:  25 	lambda:  1

N:  25 	lambda:  10

N:  25 	lambda:  25

N:  25 	lambda:  45

N:  25 	lambda:  70

N:  25 	lambda:  100

N:  25 	greedy

N:  25 	reluctant


Running simulations: 100%|██████████| 1000/1000 [00:01<00:00, 980.17it/s]



N:  40 	lambda:  1

N:  40 	lambda:  10

N:  40 	lambda:  25

N:  40 	lambda:  45

N:  40 	lambda:  70

N:  40 	lambda:  100

N:  40 	greedy

N:  40 	reluctant


Running simulations: 100%|██████████| 1000/1000 [00:01<00:00, 653.64it/s]



N:  50 	lambda:  1

N:  50 	lambda:  10

N:  50 	lambda:  25

N:  50 	lambda:  45

N:  50 	lambda:  70

N:  50 	lambda:  100

N:  50 	greedy

N:  50 	reluctant


Running simulations: 100%|██████████| 1000/1000 [00:02<00:00, 435.83it/s]



N:  100 	lambda:  1

N:  100 	lambda:  10

N:  100 	lambda:  25

N:  100 	lambda:  45

N:  100 	lambda:  70

N:  100 	lambda:  100

N:  100 	greedy

N:  100 	reluctant


Running simulations: 100%|██████████| 1000/1000 [00:14<00:00, 68.29it/s]



N:  150 	lambda:  1


KeyboardInterrupt: 

## Sparse experiments

### Setup code

In [8]:
def run_sparse_simulation(N, p, lambda_param = None, dynamic = None, nreal = 1000):
    time = np.zeros(nreal)
    min_energy = np.zeros(nreal)
    
    if dynamic == "greedy":
        for i in range(nreal):
            J, sigma = initialize_system(N, p)
            final_sigma, iterations = evolution_rule_extreme(J, np.copy(sigma), dynamic)
            time[i] = iterations
            min_energy[i] = -0.5 * final_sigma.T @ J @ final_sigma / N
    
    elif dynamic == "reluctant":
        for i in tqdm(range(nreal), desc="Running simulations"):
            J, sigma = initialize_system(N, p)
            final_sigma, iterations = evolution_rule_extreme(J, np.copy(sigma), dynamic)
            time[i] = iterations
            min_energy[i] = -0.5 * final_sigma.T @ J @ final_sigma / N
    
    else:
        for i in range(nreal):
            J, sigma = initialize_system(N, p)
            final_sigma, iterations = evolution_rule(J, np.copy(sigma), lambda_param)
            time[i] = iterations
            min_energy[i] = -0.5 * final_sigma.T @ J @ final_sigma / N

    log_time = np.log10(time)
    avg_time = np.mean(time)
    avg_min_energy = np.mean(min_energy)
    std_time = np.std(time)
    std_energy = np.std(min_energy)
    return avg_time, avg_min_energy, std_time, std_energy, log_time

def run_sparse_experiment(p_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    N_values = [25, 40, 50, 100, 150, 200, 300]
    # where p is the probability it's 0.

    results_time = np.zeros((len(N_values), len(p_values)*2))
    results_energy = np.zeros((len(N_values), len(p_values)*2))
    std_time = np.zeros((len(N_values), len(p_values)*2))
    std_energy = np.zeros((len(N_values), len(p_values)*2))
    log_time = np.zeros((len(N_values), len(p_values)*2, 1000))

    for i in range(len(N_values)):
        for j in range(len(p_values)):
            print("\nN: ", N_values[i], "\tgreedy \t p_val: ", p_values[j])
            results_time[i][j], results_energy[i][j], std_time[i, j], std_energy[i, j], log_time[i, j] = run_sparse_simulation(N_values[i], p_values[j], dynamic="greedy")
            print("\nN: ", N_values[i], "\treluctant \t p_val: ", p_values[j])
            results_time[i][len(p_values)+j], results_energy[i][len(p_values)+j], std_time[i, len(p_values)+j], std_energy[i, len(p_values)+j], log_time[i, len(p_values)+j] = run_sparse_simulation(N_values[i], p_values[j], dynamic="reluctant")

    return results_time, results_energy, std_time, std_energy, log_time

import pandas as pd
def save_sparse_results(results_time, results_energy, std_time, std_energy, log_time, name, p_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
        N_values = [25, 40, 50, 100, 150, 200, 300]

        data = []

        for i in range(results_time.shape[0]):  # Loop over N values
            for j in range(len(p_values)):
                data.append([
                    N_values[i],
                    p_values[j],
                    0,
                    results_time[i, j],
                    results_energy[i, j], 
                    std_time[i, j],
                    std_energy[i, j], 
                    log_time[i, j].tolist(), 
                    np.mean(log_time[i, j].tolist())
                ])
                data.append([
                    N_values[i],
                    p_values[j],
                    -1,
                    results_time[i, len(p_values)+j],
                    results_energy[i, len(p_values)+j], 
                    std_time[i, len(p_values)+j],
                    std_energy[i, len(p_values)+j], 
                    log_time[i, len(p_values)+j].tolist(), 
                    np.mean(log_time[i, len(p_values)+j].tolist())
                ])
        df = pd.DataFrame(data, columns=['N', 'p', 'Lambda', 'Time', 'Energy', 'std_Time', 'std_Energy', 'log_time', 'avg_log_time'])
        df.to_csv(name + '.csv', index=False)
        return df


### Sparse Gaussian (Cont.)

In [11]:
def initialize_J_gaussian_sparse(N, p):
    J = np.zeros((N, N))
    std = np.sqrt(1 / (1 - p))  # = sqrt(3) if p=2/3
    
    for i in range(N):
        for j in range(i + 1, N):
            if np.random.rand() > p:
                J_ij = np.random.normal(loc=0, scale=std) / np.sqrt(N)
                J[i, j] = J_ij
                J[j, i] = J_ij

    return J

def initialize_system(N, p):
    J = initialize_J_gaussian_sparse(N, p)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [12]:
p_vals = [0.25, 0.75]
results_time1, results_energy1, std_time1, std_energy1, log_time1 = run_sparse_experiment(p_values = p_vals)
df = save_sparse_results(results_time1, results_energy1, std_time1, std_energy1, log_time1, "data/sparse_data/j_sparsenorm2_dist", p_values = p_vals)


N:  25 	greedy 	 p_val:  0.25

N:  25 	reluctant 	 p_val:  0.25


Running simulations: 100%|██████████| 1000/1000 [00:02<00:00, 402.57it/s]



N:  25 	greedy 	 p_val:  0.75

N:  25 	reluctant 	 p_val:  0.75


Running simulations: 100%|██████████| 1000/1000 [00:01<00:00, 945.75it/s]



N:  40 	greedy 	 p_val:  0.25

N:  40 	reluctant 	 p_val:  0.25


...

Running simulations: 100%|██████████| 1000/1000 [01:52<00:00,  8.91it/s]



N:  300 	greedy 	 p_val:  0.25

N:  300 	reluctant 	 p_val:  0.25


Running simulations: 100%|██████████| 1000/1000 [14:41<00:00,  1.13it/s]



N:  300 	greedy 	 p_val:  0.75

N:  300 	reluctant 	 p_val:  0.75


Running simulations: 100%|██████████| 1000/1000 [05:53<00:00,  2.83it/s]


### Sparse Rademacher

In [6]:
def initialize_J_rademacher_sparse(N, p):
    J = np.random.rand(N, N)
    non_zero_p = (1 - p)/2
    for i in range(N):
        for j in range(i + 1, N):
            if J[i, j] < p:
                J[i, j] = 0
            elif J[i, j] < p + non_zero_p:
                J[i, j] = -1/np.sqrt(N*(1-p))
            else:
                J[i, j] = 1/np.sqrt(N*(1-p))
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0) 
    return J

def initialize_system(N, p):
    J = initialize_J_rademacher_sparse(N, p)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [7]:
p_vals = [0.25, 0.75]
results_time2, results_energy2, std_time2, std_energy2, log_time2 = run_sparse_experiment(p_values = p_vals)
df = save_sparse_results(results_time2, results_energy2, std_time2, std_energy2, log_time2, "data/sparse_data/j_sparserademacher2_dist", p_values = p_vals)

# took 114 minutes


N:  25 	greedy 	 p_val:  0.25

N:  25 	reluctant 	 p_val:  0.25


Running simulations: 100%|██████████| 1000/1000 [00:01<00:00, 868.08it/s]



N:  25 	greedy 	 p_val:  0.75

N:  25 	reluctant 	 p_val:  0.75


Running simulations: 100%|██████████| 1000/1000 [00:00<00:00, 1471.70it/s]



N:  40 	greedy 	 p_val:  0.25

N:  40 	reluctant 	 p_val:  0.25


...


N:  300 	greedy 	 p_val:  0.75

N:  300 	reluctant 	 p_val:  0.75


Running simulations:  15%|█▍        | 149/1000 [5:25:42<969:05:06, 4099.54s/it] 

Maximum iterations reached without finding a local minimum


Running simulations: 100%|██████████| 1000/1000 [5:44:43<00:00, 20.68s/it]     


IndexError: index 9 is out of bounds for axis 1 with size 4

### Sparse Irrational

In [3]:
def initialize_J_sparse_irrational(N, p):
    J = np.random.rand(N, N)
    p_neg1 = np.sqrt(2) - 1 
    p_0 = 1 - (np.sqrt(2) / 2)
    p_root2 = 1 - p_neg1 - p_0
    
    P = (p-p_0)/(1-p_0)

    values = [-1/np.sqrt(N*(1-P)), 0, np.sqrt(2)/np.sqrt(N*(1-P))]
    probs = [p_neg1, p_0, p_root2]
    
    for i in range(N):
        for j in range(i+1, N):
            if J[i, j] < P:
                J[i, j] = 0
            else:
                J[i, j] = np.random.choice(values, p=probs)
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0)

    return J

def initialize_system(N, p):
    J = initialize_J_sparse_irrational(N, p)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [5]:
#p_vals = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
p_vals = [0.75]
results_time3, results_energy3, std_time3, std_energy3, log_time3 = run_sparse_experiment(p_values = p_vals)
df = save_sparse_results(results_time3, results_energy3, std_time3, std_energy3, log_time3, "data/sparse_data/j_sparseirrational2_dist", p_values = p_vals)
# 533 m with max iter at 10^6


N:  25 	greedy 	 p_val:  0.75

N:  25 	reluctant 	 p_val:  0.75


Running simulations: 100%|██████████| 1000/1000 [00:03<00:00, 282.39it/s]



N:  40 	greedy 	 p_val:  0.75

N:  40 	reluctant 	 p_val:  0.75


Running simulations: 100%|██████████| 1000/1000 [00:08<00:00, 114.29it/s]



N:  50 	greedy 	 p_val:  0.75

N:  50 	reluctant 	 p_val:  0.75


...

Running simulations: 100%|██████████| 1000/1000 [03:08<00:00,  5.31it/s]



N:  200 	greedy 	 p_val:  0.75

N:  200 	reluctant 	 p_val:  0.75


Running simulations: 100%|██████████| 1000/1000 [1:09:08<00:00,  4.15s/it]   



N:  300 	greedy 	 p_val:  0.75

N:  300 	reluctant 	 p_val:  0.75


Running simulations: 100%|██████████| 1000/1000 [10:42<00:00,  1.56it/s]


## Sparse experiments with lambda

### Setup code

In [ ]:
def run_sparse_lambda_simulation(N, p, lambda_param = None, dynamic = None, nreal = 1000):
    time = np.zeros(nreal)
    min_energy = np.zeros(nreal)
    
    if dynamic == "greedy":
        for i in range(nreal):
            J, sigma = initialize_system(N, p)
            final_sigma, iterations = evolution_rule_extreme(J, np.copy(sigma), dynamic)
            time[i] = iterations
            min_energy[i] = -0.5 * final_sigma.T @ J @ final_sigma / N
    
    elif dynamic == "reluctant":
        for i in tqdm(range(nreal), desc="Running simulations"):
            J, sigma = initialize_system(N, p)
            final_sigma, iterations = evolution_rule_extreme(J, np.copy(sigma), dynamic)
            time[i] = iterations
            min_energy[i] = -0.5 * final_sigma.T @ J @ final_sigma / N
    
    else:
        for i in range(nreal):
            J, sigma = initialize_system(N, p)
            final_sigma, iterations = evolution_rule(J, np.copy(sigma), lambda_param)
            time[i] = iterations
            min_energy[i] = -0.5 * final_sigma.T @ J @ final_sigma / N

    log_time = np.log10(time)
    avg_time = np.mean(time)
    avg_min_energy = np.mean(min_energy)
    std_time = np.std(time)
    std_energy = np.std(min_energy)
    return avg_time, avg_min_energy, std_time, std_energy, log_time

def run_sparse_lambda_experiment(p_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    N_values = [25, 40, 50, 100, 150, 200, 300]
    lambda_values = [1, 10, 25, 45, 70, 100]
    # where p is the probability it's 0.

    results_time = np.zeros((len(N_values), len(lambda_values),len(p_values)))
    results_energy = np.zeros((len(N_values), len(lambda_values),len(p_values)))
    std_time = np.zeros((len(N_values), len(lambda_values),len(p_values)))
    std_energy = np.zeros((len(N_values), len(lambda_values),len(p_values)))
    log_time = np.zeros((len(N_values), len(lambda_values),len(p_values), 1000))

    for i in range(len(N_values)):
        for k in range(len(lambda_values)):
            for j in range(len(p_values)):
                print("\nN: ", N_values[i], "\tLambda: ", lambda_values[k], "\t p_val: ", p_values[j])
                results_time[i, k, j], results_energy[i, k, j], std_time[i, k, j], std_energy[i, k, j], log_time[i, k, j] = run_sparse_lambda_simulation(N_values[i], p_values[j], lambda_param=lambda_values[k])
           
    return results_time, results_energy, std_time, std_energy, log_time

def save_sparse_lambda_results(results_time, results_energy, std_time, std_energy, log_time, name, p_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
        N_values = [25, 40, 50, 100, 150, 200, 300]
        lambda_values = [1, 10, 25, 45, 70, 100]
        data = []
        for i in range(results_time.shape[0]):  # Loop over N values
            for k in range(len(lambda_values)):
                for j in range(len(p_values)):
                    data.append([
                        N_values[i],
                        p_values[j],
                        lambda_values[k],
                        results_time[i, k, j],
                        results_energy[i, k, j], 
                        std_time[i, k, j],
                        std_energy[i, k, j], 
                        log_time[i, k, j].tolist(), 
                        np.mean(log_time[i, k, j].tolist())
                    ])
        df = pd.DataFrame(data, columns=['N', 'p', 'Lambda', 'Time', 'Energy', 'std_Time', 'std_Energy', 'log_time', 'avg_log_time'])
        df.to_csv(name + '.csv', index=False)
        return df


### Sparse Gaussian (Cont.)

In [ ]:
def initialize_J_gaussian_sparse(N, p):
    J = np.zeros((N, N))
    std = np.sqrt(1 / (1 - p))  # = sqrt(3) if p=2/3
    
    for i in range(N):
        for j in range(i + 1, N):
            if np.random.rand() > p:
                J_ij = np.random.normal(loc=0, scale=std) / np.sqrt(N)
                J[i, j] = J_ij
                J[j, i] = J_ij

    return J

def initialize_system(N, p):
    J = initialize_J_gaussian_sparse(N, p)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [ ]:
p_vals = [0.1, 0.25, 0.4, 0.5, 0.75, 0.9]
results_time1, results_energy1, std_time1, std_energy1, log_time1 = run_sparse_lambda_experiment(p_values = p_vals)
df = save_sparse_lambda_results(results_time1, results_energy1, std_time1, std_energy1, log_time1, "data/sparse_lam_data/j_sparsenorm_dist", p_values = p_vals)


N:  25 	Lambda:  1 	 p_val:  0.1

N:  25 	Lambda:  1 	 p_val:  0.25

N:  25 	Lambda:  1 	 p_val:  0.4

N:  25 	Lambda:  1 	 p_val:  0.5

N:  25 	Lambda:  1 	 p_val:  0.75

N:  25 	Lambda:  1 	 p_val:  0.9

N:  25 	Lambda:  10 	 p_val:  0.1

N:  25 	Lambda:  10 	 p_val:  0.25

N:  25 	Lambda:  10 	 p_val:  0.4

N:  25 	Lambda:  10 	 p_val:  0.5

N:  25 	Lambda:  10 	 p_val:  0.75

N:  25 	Lambda:  10 	 p_val:  0.9

N:  25 	Lambda:  25 	 p_val:  0.1

N:  25 	Lambda:  25 	 p_val:  0.25

N:  25 	Lambda:  25 	 p_val:  0.4

N:  25 	Lambda:  25 	 p_val:  0.5

N:  25 	Lambda:  25 	 p_val:  0.75

N:  25 	Lambda:  25 	 p_val:  0.9

N:  25 	Lambda:  45 	 p_val:  0.1

N:  25 	Lambda:  45 	 p_val:  0.25

N:  25 	Lambda:  45 	 p_val:  0.4

N:  25 	Lambda:  45 	 p_val:  0.5

N:  25 	Lambda:  45 	 p_val:  0.75

N:  25 	Lambda:  45 	 p_val:  0.9

N:  25 	Lambda:  70 	 p_val:  0.1

N:  25 	Lambda:  70 	 p_val:  0.25

N:  25 	Lambda:  70 	 p_val:  0.4

N:  25 	Lambda:  70 	 p_val:  0.5

N:  25 	Lambda: 

IndexError: index 6 is out of bounds for axis 2 with size 6

### Sparse Rademacher

In [ ]:
def initialize_J_rademacher_sparse(N, p):
    J = np.random.rand(N, N)
    non_zero_p = (1 - p)/2
    for i in range(N):
        for j in range(i + 1, N):
            if J[i, j] < p:
                J[i, j] = 0
            elif J[i, j] < p + non_zero_p:
                J[i, j] = -1/np.sqrt(N*(1-p))
            else:
                J[i, j] = 1/np.sqrt(N*(1-p))
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0) 
    return J

def initialize_system(N, p):
    J = initialize_J_rademacher_sparse(N, p)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [ ]:
p_vals = [0.1, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.9]
results_time2, results_energy2, std_time2, std_energy2, log_time2 = run_sparse_lambda_experiment(p_values = p_vals)
df = save_sparse_lambda_results(results_time2, results_energy2, std_time2, std_energy2, log_time2, "data/sparse_lam_data/j_sparserademacher2_dist", p_values = p_vals)


N:  25 	Lambda:  1 	 p_val:  0.1

N:  25 	Lambda:  1 	 p_val:  0.25

N:  25 	Lambda:  1 	 p_val:  0.3

N:  25 	Lambda:  1 	 p_val:  0.4

N:  25 	Lambda:  1 	 p_val:  0.5

N:  25 	Lambda:  1 	 p_val:  0.6

N:  25 	Lambda:  1 	 p_val:  0.7

N:  25 	Lambda:  1 	 p_val:  0.75

N:  25 	Lambda:  1 	 p_val:  0.8

N:  25 	Lambda:  1 	 p_val:  0.9

N:  25 	Lambda:  10 	 p_val:  0.1

N:  25 	Lambda:  10 	 p_val:  0.25

N:  25 	Lambda:  10 	 p_val:  0.3

N:  25 	Lambda:  10 	 p_val:  0.4

N:  25 	Lambda:  10 	 p_val:  0.5


/var/folders/6w/_v5vk_hx361283vfrn4nr6700000gn/T/ipykernel_64076/1858413932.py:26: RuntimeWarning: divide by zero encountered in log10
  log_time = np.log10(time)



N:  25 	Lambda:  10 	 p_val:  0.6

N:  25 	Lambda:  10 	 p_val:  0.7

N:  25 	Lambda:  10 	 p_val:  0.75

N:  25 	Lambda:  10 	 p_val:  0.8

N:  25 	Lambda:  10 	 p_val:  0.9

N:  25 	Lambda:  25 	 p_val:  0.1

N:  25 	Lambda:  25 	 p_val:  0.25

N:  25 	Lambda:  25 	 p_val:  0.3

N:  25 	Lambda:  25 	 p_val:  0.4

N:  25 	Lambda:  25 	 p_val:  0.5

N:  25 	Lambda:  25 	 p_val:  0.6

N:  25 	Lambda:  25 	 p_val:  0.7

N:  25 	Lambda:  25 	 p_val:  0.75

N:  25 	Lambda:  25 	 p_val:  0.8

N:  25 	Lambda:  25 	 p_val:  0.9

N:  25 	Lambda:  45 	 p_val:  0.1

N:  25 	Lambda:  45 	 p_val:  0.25

N:  25 	Lambda:  45 	 p_val:  0.3

N:  25 	Lambda:  45 	 p_val:  0.4

N:  25 	Lambda:  45 	 p_val:  0.5

N:  25 	Lambda:  45 	 p_val:  0.6

N:  25 	Lambda:  45 	 p_val:  0.7

N:  25 	Lambda:  45 	 p_val:  0.75

N:  25 	Lambda:  45 	 p_val:  0.8

N:  25 	Lambda:  45 	 p_val:  0.9

N:  25 	Lambda:  70 	 p_val:  0.1

N:  25 	Lambda:  70 	 p_val:  0.25

N:  25 	Lambda:  70 	 p_val:  0.3

N:  25 	Lambd

### Sparse Irrational

In [ ]:
def initialize_J_sparse_irrational(N, p):
    J = np.random.rand(N, N)
    p_neg1 = np.sqrt(2) - 1 
    p_0 = 1 - (np.sqrt(2) / 2)
    p_root2 = 1 - p_neg1 - p_0
    
    P = (p-p_0)/(1-p_0)

    values = [-1/np.sqrt(N*(1-P)), 0, np.sqrt(2)/np.sqrt(N*(1-P))]
    probs = [p_neg1, p_0, p_root2]
    
    for i in range(N):
        for j in range(i+1, N):
            if J[i, j] < P:
                J[i, j] = 0
            else:
                J[i, j] = np.random.choice(values, p=probs)
            J[j, i] = J[i, j]
    np.fill_diagonal(J, 0)

    return J

def initialize_system(N, p):
    J = initialize_J_sparse_irrational(N, p)
    sigma = np.random.choice([-1, 1], size=N)
    return J, sigma

In [ ]:
p_vals = [0.3, 0.4, 0.5, 0.75, 0.9]
results_time3, results_energy3, std_time3, std_energy3, log_time3 = run_sparse_lambda_experiment(p_values = p_vals)
df = save_sparse_lambda_results(results_time3, results_energy3, std_time3, std_energy3, log_time3, "data/sparse_lam_data/j_sparseirrational_dist", p_values = p_vals)


N:  25 	Lambda:  1 	 p_val:  0.3

N:  25 	Lambda:  1 	 p_val:  0.4

N:  25 	Lambda:  1 	 p_val:  0.5

N:  25 	Lambda:  1 	 p_val:  0.75

N:  25 	Lambda:  1 	 p_val:  0.9

N:  25 	Lambda:  10 	 p_val:  0.3

N:  25 	Lambda:  10 	 p_val:  0.4

N:  25 	Lambda:  10 	 p_val:  0.5

N:  25 	Lambda:  10 	 p_val:  0.75

N:  25 	Lambda:  10 	 p_val:  0.9

N:  25 	Lambda:  25 	 p_val:  0.3

N:  25 	Lambda:  25 	 p_val:  0.4

N:  25 	Lambda:  25 	 p_val:  0.5

N:  25 	Lambda:  25 	 p_val:  0.75


/var/folders/6w/_v5vk_hx361283vfrn4nr6700000gn/T/ipykernel_64076/1858413932.py:26: RuntimeWarning: divide by zero encountered in log10
  log_time = np.log10(time)



N:  25 	Lambda:  25 	 p_val:  0.9

N:  25 	Lambda:  45 	 p_val:  0.3

N:  25 	Lambda:  45 	 p_val:  0.4

N:  25 	Lambda:  45 	 p_val:  0.5

N:  25 	Lambda:  45 	 p_val:  0.75

N:  25 	Lambda:  45 	 p_val:  0.9

N:  25 	Lambda:  70 	 p_val:  0.3

N:  25 	Lambda:  70 	 p_val:  0.4

N:  25 	Lambda:  70 	 p_val:  0.5

N:  25 	Lambda:  70 	 p_val:  0.75

N:  25 	Lambda:  70 	 p_val:  0.9

N:  25 	Lambda:  100 	 p_val:  0.3

N:  25 	Lambda:  100 	 p_val:  0.4

N:  25 	Lambda:  100 	 p_val:  0.5

N:  25 	Lambda:  100 	 p_val:  0.75

N:  25 	Lambda:  100 	 p_val:  0.9

N:  40 	Lambda:  1 	 p_val:  0.3

N:  40 	Lambda:  1 	 p_val:  0.4

N:  40 	Lambda:  1 	 p_val:  0.5

N:  40 	Lambda:  1 	 p_val:  0.75

N:  40 	Lambda:  1 	 p_val:  0.9

N:  40 	Lambda:  10 	 p_val:  0.3

N:  40 	Lambda:  10 	 p_val:  0.4

N:  40 	Lambda:  10 	 p_val:  0.5

N:  40 	Lambda:  10 	 p_val:  0.75

N:  40 	Lambda:  10 	 p_val:  0.9

N:  40 	Lambda:  25 	 p_val:  0.3

N:  40 	Lambda:  25 	 p_val:  0.4

N:  40 	Lambda